## Imports

In [ ]:
from __future__ import print_function
%matplotlib inline

import math
import os
import cv2
import numpy as np
import keras
from keras.models import Sequential, load_model, Model
from keras.layers import Cropping2D, Lambda, Input, BatchNormalization, Concatenate, concatenate
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpim

## Utilities

In [ ]:
def get_samples_list(sampledir, exclude=[], d_range=[]):
    samples = []
    labelcount = []
    for fname in os.listdir(sampledir):
        if fname[-4:]=='.png' and not fname[0] in exclude:
            d = int(fname.split('__')[-1].split('_')[0])
            if len(d_range) > 0 and (d < d_range[0] or d > d_range[1]):
                continue
            label = int(fname[0])
            samples.append((os.path.join(sampledir, fname), label))
            while len(labelcount) < label+1:
                labelcount += [0]
            labelcount[label] += 1
    print('label counts:')
    for i in range(len(labelcount)):
        print('{}: {}'.format(i, labelcount[i]))
    print('total: {}'.format(len(samples)))
    return samples

def augment_samples_list(samples, mult=[1, 1, 1], tx=[0, 1], ty=[0, 1]):
    augs = []
    for sample in samples:
        x = mult[sample[1]]-1
        for i in range(x):
            shiftx = np.random.randint(tx[0], tx[1]+1)
            shifty = np.random.randint(ty[0], ty[1]+1)
            sample_aug = [sample[0], sample[1], shiftx, shifty]
            augs.append(sample_aug)
    return samples + augs

def filter_samples_by_distance(samples, drange=[0, 100]):
    filtered_samples = []
    for sample in samples:
        fname = sample[0].split('/')[-1]
        dee = int(fname.split('_')[2])
        if drange[0] < dee < drange[1]:
            filtered_samples.append(sample)
    return filtered_samples

def datagen(samples, batch_size=32, n_class=4, grey=False):
    n_samples = len(samples)
    while True:
        samples = shuffle(samples)
        for offset in range(0, n_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            labels = []
            for batch_sample in batch_samples:
                im = mpim.imread(batch_sample[0])
                if grey:
                    im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
#                     im = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
#                     im = im[:, :, 2]
                if len(batch_sample) > 2:
                    tx = batch_sample[2]
                    ty = batch_sample[3]
                    nrow, ncol = im.shape[:2]
                    T = np.float32([[1, 0, tx], [0, 1, ty]])
                    im = cv2.warpAffine(im, T, (ncol, nrow))
                images.append(im)
                labels.append(batch_sample[1])
                
            images = np.array(images)
            if grey:
                newshape = [x for x in images.shape] + [1]
                images = np.reshape(images, newshape)
            labels = keras.utils.to_categorical(np.array(labels), num_classes=n_class)
            
            yield shuffle(images, labels)
            
def count_sample_distro(samples):
    label_counts = []
    for sample in samples:
        label = sample[1]
        while len(label_counts) < label+1:
            label_counts += [0]
        label_counts[label] += 1
        
    total_count = sum(label_counts)
    max_count = max(label_counts)
    print('label counts, proportion, mult')
    for i, label in enumerate(label_counts):
        print('{} {:5.3f} {:6.2f}'.format(label, 1.0*label/total_count, 1.0*max_count/label))
    print('total', total_count)
    return label_counts

def get_samples_list_recursive(sample_root, exclude=[]):
    fnames = os.listdir(sample_root)
    samples = []
    for fname in fnames:
        if not (fname[0] in exclude) and fname[-4:] == '.png':
            samples.append([os.path.join(sample_root, fname), int(fname[0])])
        elif os.path.isdir(os.path.join(sample_root, fname)):
            new_root = os.path.join(sample_root, fname)
            add_samples = get_samples_list_recursive(new_root, exclude)
            samples += add_samples
    return samples            

def test_exhaustive(samples, model='model.h5', batch_size=32, grey=False):
    model = load_model(model)
    confusion = np.zeros((3, 3))
    
    N_bat = len(samples)//batch_size + 1
    for offset in range(0, len(samples), batch_size):
        batch_number = offset//batch_size+1
        if batch_number % 10 == 0:
            print('batch', batch_number, 'of', N_bat)
        labels = []
        imgs = []
        for sample in samples[offset:offset+batch_size]:
            img = mpim.imread(sample[0])
            if grey:
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#                 img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            labels.append(sample[1])

            if len(sample) > 2:
                tx = sample[2]
                ty = sample[3]
                T = np.float32([[1, 0, tx], [0, 1, ty]])
                nrow, ncol = img.shape[:2]
                img = cv2.warpAffine(img, T, (ncol, nrow))

            imgs.append(img)
        imgs = np.array(imgs)
        if grey:
            newshape = [x for x in imgs.shape] + [1]
            imgs = np.reshape(imgs, newshape)
        preds = model.predict(imgs)
        pred_labels = [int(np.argmax(pred)) for pred in preds]

        for label, pred_label in zip(labels, pred_labels):
            confusion[label][pred_label] += 1

    return confusion

## Sample generation test

In [ ]:
samples = get_samples_list_recursive('samples', exclude=['3'])
# samples = augment_samples_list(samples, mult=[1, 22, 4], tx=[-50, 51])
_ = count_sample_distro(samples)

samples = filter_samples_by_distance(samples, [0, 10])
samples = augment_samples_list(samples, mult=[1, 18, 2], tx=[-50, 50], ty=[0, 1])
_ = count_sample_distro(samples)

In [ ]:
samplegen = datagen(samples, batch_size=10, n_class=3, grey=True)

In [ ]:
imgs, labels = next(samplegen)

grey = False
cropx = 50
cropy = 0

print(imgs[0].shape)

if imgs[0].shape[-1] < 3:
    grey = True
ncols = 5
for i, img in enumerate(imgs):
    if i % ncols == 0:
        plt.figure(figsize=(12, 12))
    plt.subplot(1, ncols, (i%ncols)+1)
    img = img[cropy:-cropy-1, cropx:-cropx-1]
    if grey:
        plt.imshow(np.squeeze(img), cmap='gray')
    else:
        plt.imshow(img)
#     plt.gca().set_xlim([0+50, 800-50])
#     plt.gca().set_ylim([0+100, 600-100])
    plt.gca().set_title(labels[i])

## Net definitions

In [ ]:
def net_multiscale():
    input_tensor = Input(shape=(None, None, 3), name="input_tensor") # three channels, any size
    
    # monoscale layers
    conv_com1 = Conv2D(16, (3, 3), activation='relu')(input_tensor)
    batnorm1 = BatchNormalization()(conv_com1)
    maxpool1 = MaxPooling2D()(batnorm1)
    conv_com2 = Conv2D(32, (3, 3), activation='relu')(maxpool1)
    batnorm2 = BatchNormalization()(conv_com2)
    conv_com3 = Conv2D(64, (3, 3), activation='relu')(batnorm2)
    batnorm3 = BatchNormalization()(conv_com3)
    conv_com4 = Conv2D(128, (3, 3), activation='relu')(batnorm3)
    batnorm4 = BatchNormalization()(conv_com4)
    maxpool2 = MaxPooling2D()(batnorm4)
    dropout1 = Dropout(0.2)(maxpool2)
    
    # multiscale layers
    conv_sc3_1 = Conv2D(128, (3, 3), activation='relu')(dropout1)
    conv_sc3_2 = Conv2D(256, (3, 3), activation='relu')(conv_sc3_1)
    gap_sc3 = GlobalAveragePooling2D()(conv_sc3_2)
    
    conv_sc5_1 = Conv2D(128, (5, 5), activation='relu')(dropout1)
    conv_sc5_2 = Conv2D(256, (5, 5), activation='relu')(conv_sc5_1)
    gap_sc5 = GlobalAveragePooling2D()(conv_sc5_2)
    
    conv_sc7_1 = Conv2D(128, (7, 7), activation='relu')(dropout1)
    conv_sc7_2 = Conv2D(256, (7, 7), activation='relu')(conv_sc7_1)
    gap_sc7 = GlobalAveragePooling2D()(conv_sc7_2)
    
    concat = concatenate([gap_sc3, gap_sc5, gap_sc7])
    
    # dense layers
    dense1 = Dense(512, activation='relu')(concat)
    dropout2 = Dropout(0.2)(dense1)
    dense2 = Dense(256, activation='relu')(dense1)
    logit = Dense(3, activation='softmax')(dense2)
    
    model = Model(inputs=input_tensor, outputs=logit)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model    

In [ ]:
netms = net_multiscale()
netms.summary()

In [ ]:
def net_nvidia(n_class=4):
    model = Sequential()
    
    model.add(Cropping2D(((0, 0), (250, 250)), input_shape=(600, 800, 3)))
    
    model.add(Conv2D(24, kernel_size=(5, 5), strides=(2, 2)))
    model.add(Activation('relu')) # the paper doesn't mention activation function, but isn't that needed?
    
    model.add(Conv2D(36, kernel_size=(5, 5), strides=(2, 2)))
    model.add(Activation('relu')) 
    
    model.add(Conv2D(48, kernel_size=(5, 5), strides=(2, 2)))
    model.add(Activation('relu')) 
    
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1)))
    model.add(Activation('relu')) 
    
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1)))
    model.add(Dropout(0.30))
    model.add(Activation('relu')) 
    
    model.add(Flatten())
    
    model.add(Dense(100))
    model.add(Activation('relu'))
    
    model.add(Dense(50))
    model.add(Activation('relu'))
    
    model.add(Dense(10))
    model.add(Activation('relu'))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(lr=1e-6), metrics=['accuracy'])
    
    return model

In [ ]:
def net_simple(n_class=4):
    model = Sequential()
    
    model.add(Cropping2D((100, 250), input_shape=(600, 800, 3)))
    
    model.add(Conv2D(8, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(16, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(lr=1e-6), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def net2(n_class=3):
    model = Sequential()
    model.add(Cropping2D((100, 250), input_shape=(600, 800, 3)))
    
    model.add(Conv2D(16, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(256, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def netgrey(n_class=3):
    model = Sequential()
    model.add(Cropping2D((100, 50), input_shape=(600, 800, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2))) # scale by half
    model.add(Lambda(lambda x: 2.0*x-1.0))
    
    model.add(Conv2D(8, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(16, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))

    model.add(Flatten())
    
    model.add(Dropout(0.2))
    
#     model.add(Dense(256))
#     model.add(Activation('relu'))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def netgrey2(n_class=3):
    model = Sequential()
    model.add(Cropping2D((0, 50), input_shape=(600, 800, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2))) # scale by half
    model.add(Lambda(lambda x: 2.0*x-1.0))
    
    model.add(Conv2D(8, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(16, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))

    model.add(Flatten())
    
    model.add(Dropout(0.2))
    
#     model.add(Dense(256))
#     model.add(Activation('relu'))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def netungrey(n_class=3):
    model = Sequential()
    model.add(Cropping2D((0, 50), input_shape=(600, 800, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2))) # scale by half
    model.add(Lambda(lambda x: 2.0*x-1.0))
    
    model.add(Conv2D(8, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))
    
    model.add(Conv2D(16, kernel_size=(5, 5)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Activation('relu'))

    model.add(Flatten())
    
    model.add(Dropout(0.2))
    
#     model.add(Dense(256))
#     model.add(Activation('relu'))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    
    model.add(Dense(16))
    model.add(Activation('relu'))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def train(model, samples, batch_size=32, epochs=1, grey=False):        
    train_samples, valid_samples = train_test_split(samples, test_size=0.25)
    train_gen = datagen(train_samples, batch_size=batch_size, n_class=3, grey=grey)
    valid_gen = datagen(valid_samples, batch_size=batch_size, n_class=3, grey=grey)
    
    train_step = math.ceil(len(train_samples)/batch_size)
    valid_step = math.ceil(len(valid_samples)/batch_size)
    
    history = model.fit_generator(train_gen, steps_per_epoch=train_step,
                                  validation_data=valid_gen, validation_steps=valid_step,
                                  epochs=epochs, verbose=1)
    model.save('model.h5')
    return history

## Model training

In [ ]:
samples = get_samples_list_recursive('samples', exclude=['3'])
samples = augment_samples_list(samples, mult=[1, 21, 3], tx=[-50, 50], ty=[0, 1])
_ = count_sample_distro(samples)

In [ ]:
model = net_multiscale()
h = train(model, samples, batch_size=32, epochs=1, grey=False)

In [ ]:
samples = get_samples_list_recursive('samples', exclude=['3'])
samples = augment_samples_list(samples, mult=[1, 21, 3], tx=[-50, 50], ty=[0, 1])
_ = count_sample_distro(samples)
model = load_model('model.h5')
# model.summary()
h = train(model, samples, batch_size=32, epochs=4, grey=True)

In [ ]:
# Further training on only close-range data
samples = get_samples_list_recursive('samples', exclude=['3'])
samples = augment_samples_list(samples, mult=[1, 21, 3], tx=[-50, 50], ty=[0, 1])
xsamples = filter_samples_by_distance(samples, drange=[0, 15])
_ = count_sample_distro(xsamples)

In [ ]:
model = load_model('model.h5')
h = train(model, xsamples, batch_size=32, epochs=4, grey=True)

## Testing result

In [ ]:
samples = get_samples_list_recursive('samples', exclude=['3'])
samples = augment_samples_list(samples, mult=[1, 10, 2], tx=[-50, 51])
_ = count_sample_distro(samples)

In [ ]:
conf = test_exhaustive(samples, model='model.h5', batch_size=64, grey=True)

In [ ]:
plt.imshow(conf, cmap='gray')

In [ ]:
print(conf)
accuracy = sum([conf[i, i] for i in range(len(conf))])/sum(sum(conf))
print('accuracy', accuracy)

for i in range(len(conf)):
    print('class', i)
    precision = conf[i, i] / (sum(conf[:, i])) # true +ve / (true +ve + false +ve)
    recall = conf[i, i] / (sum(conf[i, :])) # true +ve / (true+ve + false -ve)
    print('precision', precision)
    print('recall', recall)
    print()

In [ ]:
samples = get_samples_list('samples', exclude=['3'])
samples = augment_samples_list(samples, mult=[1, 150, 5])

In [ ]:
samples_gen = datagen(samples, batch_size=32, n_class=3)

In [ ]:
model = load_model('model_simple92.h5')

In [ ]:
imgs, labels = next(samples_gen)
preds = model.predict(imgs, len(imgs), verbose=0)
# print(preds.shape)
# for pred, label in zip(preds, labels):
#     print(np.argmax(label), np.argmax(pred), pred)

good = 0
for label, pred in zip(labels, preds):
    if np.argmax(label) == np.argmax(pred):
        good += 1
print('accuracy', good/len(labels))

In [ ]:
imx = mpim.imread('samples/0__21_1579319374_884872913.png')
imx = np.expand_dims(imx, axis=0)
print(imx.shape)
p = model.predict(imx)
print(np.argmax(p[0]), p[0])

In [ ]:
for i in range(0, len(imgs), 3):
    plt.figure(figsize=(16, 16))
    for j in range(3):
        if i + j < len(imgs):
            plt.subplot(1, 3, j+1)
            plt.imshow(imgs[i+j])
            real_class = np.argmax(labels[i+j])
            detect_class = np.argmax(preds[i+j])
            confidence = np.max(preds[i+j])
            plt.gca().set_title('rel {} det {} con {:5.3f}'\
                                .format(real_class, detect_class, confidence))